In [1]:
import os

from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

from config_notebooks import *

In [2]:
os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = AZURE_SEARCH_SERVICE_ENDPOINT
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = AZURE_SEARCH_INDEX_NAME
os.environ["AZURE_AI_SEARCH_API_KEY"] = AZURE_SEARCH_ADMIN_KEY

In [3]:
retriever_09 = AzureAISearchRetriever(
    content_key="content", top_k=3, index_name=AZURE_SEARCH_INDEX_NAME,
    filter="sourcepage eq 'GEOG_CLASS11.pdf'"
)

In [4]:
os.environ["AZURE_OPENAI_API_KEY"] =AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] =AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = "2023-05-15"


In [5]:
from langchain.chat_models import AzureChatOpenAI

In [6]:
llm = AzureChatOpenAI(azure_deployment=AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID,temperature=0)

c:\Ambarish\text2sql\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [8]:
from langchain.chains import RetrievalQA

In [9]:
from langchain.chains import RetrievalQA
from langchain_core.tools import Tool

from langchain_core.pydantic_v1 import BaseModel, Field

In [10]:
class DocumentInput(BaseModel):
    question: str = Field()

In [11]:
FILE_NAME_02 = "CHAP02"
FILE_NAME_09 = "CHAP09"

In [12]:
tool_09 = Tool(
    args_schema=DocumentInput,
            name=FILE_NAME_09,
            description=f"useful when you want to answer questions about {FILE_NAME_09}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever_09),
        )

In [13]:
tools = [tool_09]

In [14]:
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [15]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [16]:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

c:\Ambarish\text2sql\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [17]:
agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
action_input = "What is typhi in Chap09 ?"

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `CHAP09` with `{'question': 'What is typhi?'}`


{'query': 'What is typhi?', 'result': 'Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water. Proper sanitation, vaccination, and antibiotics are key measures in preventing and treating typhoid fever.'}Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water. Proper sanitation, vaccination, and antibiotics are key measures in preventing and treating typhoid fever.

> Finished chain.


{'input': 'What is typhi in Chap09 ?',
 'output': 'Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water. Proper sanitation, vaccination, and antibiotics are key measures in preventing and treating typhoid fever.'}

In [20]:
from langchain.agents import AgentType, initialize_agent

In [21]:
agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

agent({"input": action_input})

c:\Ambarish\text2sql\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Ambarish\text2sql\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `CHAP09` with `{'question': 'What is typhi?'}`


{'query': 'What is typhi?', 'result': 'Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water. If you need more specific information or have other questions, feel free to ask!'}Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water.

> Finished chain.


{'input': 'What is typhi in Chap09 ?',
 'output': 'Typhi refers to *Salmonella enterica* serotype Typhi, a bacterium that causes typhoid fever. Typhoid fever is a serious and potentially life-threatening illness characterized by high fever, weakness, stomach pains, headache, and loss of appetite. It is typically spread through contaminated food and water.'}